In [ ]:
import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install requests
from bs4 import BeautifulSoup
import requests
from urllib import request
import os.path
import pandas as pd
from googletrans import Translator
from pathlib import Path

#Import NLTK and Texts
import nltk
nltk.download('gutenberg')
nltk.download('averaged_perceptron_tagger')
#Command All Matplotlib Graphs to Appear in Inline in Notebook
# %matplotlib inline

# https://www.gutenberg.org/cache/epub/3600/pg3600-images.html
# https://gutendex.com/
# https://www.scrapingbee.com/blog/python-web-scraping-beautiful-soup/
# https://www.geeksforgeeks.org/find-the-text-of-the-given-tag-using-beautifulsoup/

In [ ]:
############################# EDIT HERE ##########
## EDITING BEGINS
book_url = requests.get("https://www.gutenberg.org/cache/epub/29336/pg29336-images.html")
file_name_txt = 'may_dict.txt'
file_name_csv = 'may_dict.csv'

# add additional stop words here as needed
stop_words = 'maxime', 'häckerling', 'gretchen', 'margarete', 'mephistopheles', 'wolfgang', 'johann', 'goethe', 'tragödie', 'erster', 'baubo', 'xenien', 'jahre', 'o', 't', 's', 'großes', 'fakultät', 'szene', 'paragraphos', 'lara', 'herminen', 'Weasleyzwilling', 'Wetter', 'namensliste', 'Klassenzimmer', 'wände', 'zahn', 'Zucker', 'Freund', 'Papier', 'Abendessen', 'Pflanze', 'Recht', 'Zeitungsartikel', 'klassenzimmern', 'Fuß', 'Schule', 'Hand', 'Tier', 'künste', 'Nachmittag', 'Tier', 'Kreatur', 'Lehrer', 'Lehrerin', 'Frühstück', 'Porträt', 'Muggelfamilie', 'peev', 'Arbeit', 'leeren', 'Schüler', 'Zombie', 'emmerich', 'Aconitum', 'leg', 'Affodill', 'weasley', 'Potter', 'haben', 'Tages', 'klein', 'bringen', 'Meister', 'erst', 'in', 'vor', 'heißen', 'Tee', 'rasn', 'trinken', 'können', 'ohne', 'sehen', 'spät', 'möchten', 'unfair', 'Paar', 'jetzt', 'Kunst', 'kessel', 'von', 'Poltergeist', 'kommen', 'schick', 'Information', 'Gringott', 'studieren', 'Juli', 'Granger', 'mitkommen', 'manchmal', 'Prinz', 'afrikanisch', 'Tagesprophete', 'quirrells', 'immer', 'nur', 'mit', 'da', 'oft', 'neu', 'gehen', 'argus', 'Filch', 'wo', 'wie', 'ja', 'wenn', 'vielleicht', 'jung', 'alt', 'morgen', 'sehr', 'für', 'klug', 'Freitag', 'freitags', 'hermin', 'Fang', 'Tag', 'Waldes', 'zusammen', 'Mittwoch', 'hier', 'dann', 'denn', 'halb', 'schon', 'allein', 'gut', 'bald', 'lang', 'bis', 'Schuljahresbeginn', 'aber', 'heute', 'Lust', 'Slytherin', 'tags', 'Post', 'Eule', 'gerne', 'Frühstücks', 'kalt', 'warum', 'dich', 'miserabel', 'wild', 'Wild', 'Woche', 'sir', 'extra', 'gern', 'Bruder', 'dort', 'groß', 'Moment', 'und', 'oder', 'Name', 'Professorin', 'Professor', 'Katze', 'Wort', 'Tunnel', 'Tische', 'Thema', 'Wasser', 'Eisenhut', 'Buch', 'Korridor', 'Freitagnachmittag', 'Stapel', 'Teleskope', 'Binn'

## EDITING ENDS
##################################################

In [ ]:
if book_url.status_code != 200:
	print("Error fetching page")
	exit()
else:
	content = book_url.content
# print(content)

soup = BeautifulSoup(book_url.content, 'html.parser')

# print(soup.title) # title w HTML
# print(soup.title.string) # title as string (no HTML)
# print(soup.get_text()) all text (no HTML)

data = [element.text for element in soup.find_all("p")]
bookstr = ' '.join(data)

# data cleaning - tokenize
from functions.datacleaning import dataCleaning
dc=dataCleaning(bookstr)
clean_tokens = dc.textcleaner()

typical_gutenberg_stop_words = 'german', 'language', 'may', 'author', 'updated', 'recently', 'most', 'ebook', 'june', 'date', 'release', 'title', 'january', 'february', 'march', 'april', 'july', 'august', 'september', 'october', 'november', 'december'

token_list_gutenberg = [w for w in clean_tokens if not w in typical_gutenberg_stop_words]
token_list = [w for w in token_list_gutenberg if not w in stop_words]
data_dir = os.path.join("/Users/thagen/Documents/github-hagen668/deutsch-book-quiz/data/")

from googletrans import Translator

# translation prep
translator = Translator()
flashcards_en = []
flashcards_de = []

# translation execution
for translation in token_list:
    result = translator.translate(translation, src='de', dest='en')
    flashcards_de.append(result.origin)
    flashcards_en.append(result.text)

# clean up english translations
flashcards_en = [s.replace('to ', '') for s in flashcards_en] 

# German-English dictionary created here
flashcard_dict = dict(zip(flashcards_de, flashcards_en))


#### Data exports
file_save_txt = os.path.join(data_dir, file_name_txt)

### Export dictionary to .txt file
# import necessary packages
import json
import codecs

# save as txt
with codecs.open(file_save_txt, 'w', encoding='utf-8') as convert_file:
     convert_file.write(json.dumps(flashcard_dict, ensure_ascii=False))
        
        
### Export dicitonary to .csv file
df_dict = pd.DataFrame(list(flashcard_dict.keys()), columns=['keys'])
df_dict['values'] = pd.Series(list(flashcard_dict.values()))

file_save_csv = os.path.join(data_dir, file_name_csv)

# "ensure_ascii=False" persists characters such as umlautes.
# save as csv
df_dict.to_csv(file_save_csv, sep=',', encoding='utf-8', index=False, header=False)

In [ ]:
#### View final word count of work
# import collections
# od = collections.OrderedDict(sorted(flashcard_dict.items()))
# od
# len(od)